# Creating grid to municipality matrix

## Using number of buidlings
The files were saved in the notebook `02.0_preprocessing.ipynb`

In [ ]:
%load_ext jupyter_black
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [ ]:
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/input/"
)

In [ ]:
# reading intersection csv
phl_bld_centroid_municip_intersection_count = pd.read_csv(input_dir / "phl_hotosm_bld_centroid_municip_intersection_count.csv")
phl_bld_centroid_grid_count = pd.read_csv(input_dir / "phl_hotosm_bld_centroid_grid_count.csv")

In [ ]:
phl_bld_grid_matrix = phl_bld_centroid_municip_intersection_count.merge(
    phl_bld_centroid_grid_count, on="id", suffixes=("_x", None)
)
phl_bld_grid_matrix.columns

In [ ]:
phl_bld_grid_matrix["weight"] = np.where(
    phl_bld_grid_matrix["numbuildings"] == 0,
    0,
    phl_bld_grid_matrix["numbuildings_x"]
    / phl_bld_grid_matrix["numbuildings"],
)
phl_bld_grid_matrix["weight"].describe()

In [ ]:
phl_bld_grid_matrix[phl_bld_grid_matrix["Centroid_x"] == "118.4E_9.7N"]

In [ ]:
phl_bld_grid_matrix_sel = phl_bld_grid_matrix[
    ["ADM3_PCODE", "ADM3_EN", "id", "Centroid", "weight"]
]
phl_bld_grid_matrix_sel

In [ ]:
phl_bld_grid_matrix_sel.to_csv(
    input_dir / "phl_bld_grid_to_municip_matrix.csv"
)